In [2]:
import csv
import cv2
import numpy as np
import os
import datetime

# Load face recognizer model and face cascade classifier
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# Define font for displaying text on image
font = cv2.FONT_HERSHEY_SIMPLEX

# Define dictionary of names and their corresponding IDs
names = {
    0: 'None',
    1: 'Shantam',
    2: 'Saksham',
    3: 'Shreyansh',
    4: 'Sanskar',
    5: 'Rajat'
}

# Initialize attendance dictionary to keep track of presence
attendance = {}

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

# Define minimum window size to be recognized as a face
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    # Process each detected face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Recognize the face and get its ID and confidence
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        # If confidence is less than 50%, mark the person present
        if confidence < 70:
            name = names[id]
            attendance[name] = True
            cv2.putText(img, name, (x+5, y-5), font, 1, (255, 255, 255), 2)
        else:
            name = 'unknown'
            cv2.putText(img, name, (x+5, y-5), font, 1, (255, 255, 255), 2)

        cv2.putText(img, str(round(100 - confidence)) + '%', (x+5, y+h-5), font, 1, (255, 255, 0), 1)

    # Display the image
    cv2.imshow('camera', img)

    # Stop detection if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Write attendance data to CSV file
with open('attendance.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Present', 'Date', 'Time'])
    for name, present in attendance.items():
        writer.writerow([name, present, datetime.datetime.now().date(), datetime.datetime.now().time()])

# Clean up
cam.release()
cv2.destroyAllWindows()
